<img src="https://spark.apache.org/images/spark-logo-trademark.png"
     align="right"
     width="30%">

## DataFrames (Spark)

In the previous notebook we manipulated and queried a large parquet file by splitting it up into many Pandas dataframes with the concurrent.futures API.  This was straightforward, but required us to be clever to implement various parallel algorithms.  Fortunately, for certain classes of algorithms various big data projects have already done this work for us.  Dealing with tabular data like the data in our NYC-Taxi dataset is well covered by many systems like SQL databases, Spark, and Dask.dataframes.

At the end of the last notebook we saw that we could use Pandas-like syntax to accomplish the same result with less programming.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://schedulers:7077').getOrCreate()
spark

In [ ]:
columns = ['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount']

df = (spark.read.parquet('s3a://dask-data/nyc-taxi/nyc-2015.parquet')
           .select(*columns))

In [ ]:
df.agg({'passenger_count': 'sum'}).collect()

In [ ]:
df.agg({'passenger_count': 'avg'}).collect()

In [ ]:
df.groupby('passenger_count').agg({'*': 'count'}).collect()

## Going further

In this notebook we will explore this API and this dataset further.  We will determine how well people tip based on the number of passengers in a cab.  To do this we will accomplish the following:

1.  Remove rides with zero fare
2.  Add a new column `tip_fraction` that is equal to the ratio of the tip to the fare
3.  Group by the `passenger_count` column and take the mean of the `tip_fraction` column.

You may want to refer to these resources to help you with the Spark DataFrame API

-  https://spark.apache.org/docs/latest/api/python/pyspark.sql.html
-  https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

And refer to the [Spark UI](../../../9070) for feedback.

### How to remove rows

In Spark you can filter rows by a boolean expression like the following:

```python
df = df.filter(df.name == 'Alice')
```

### How to make new columns

In Pandas you can create a new column using Python's setitem syntax like the following:

```python
df = df.withColumn('z', df.x + df.y)
```

### How to do groupby-aggregations

In Pandas you can do a groupby-aggregation by using the `groupby` method, followed by a column name an aggregation method like the following:

```python
df.groupBy(df.name).agg({'column-name': 'avg'})
```

When you want to collect the result of your computation, finish with the `.collect()` method.

## Exercise

Use the operations above to find out how well New Yorkers tip based on the number of passengers in the cab.  Be sure to filter out rides with zero fare first to get a good result.

In [ ]:
%load solutions/nyc-taxi-dataframes-spark.py

### Additional exercises

If you're already experienced with Pandas then we recommend also looking at the following questions:

1. How well do New Yorkers tip as a function of the hour of day and the day of the week?
2.  Investiate the `payment_type` column.  See how well each of the payment types correlate with the `tip_fraction`.  Did you find anything interesting?  Any guesses on what the different payment types might be?  If you're interested you may be able to find more information on the [NYC TLC's website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)
3.  How quickly can you get the data for a particular day of the year?  How about for a particular hour of that day?
